In [42]:
%reload_ext autoreload
%autoreload 2

import os
import io
import ast
import json
import importlib 
import itertools
import collections
import pprint

import pymongo

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend, ColumnDataSource, FactorRange
from bokeh.palettes import all_palettes
from bokeh.transform import factor_cmap

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

from climatools.atm.absorbers import *

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew

from climatools.cliradlw.utils import *
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.dataset import *

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

from IPython import display

In [43]:
output_notebook()

Loading BokehJS ...

In [44]:
MOL = 'co2'

In [45]:
ATMPRO = dict(mls=294, saw=257, trp=300)

In [46]:
COMMITNUMBER = '1013f91'

In [47]:
client = pymongo.MongoClient('localhost', 27017)

In [48]:
def show_html(s):
    display.display(display.HTML(s))

def show_markdown(s):
    display.display(display.Markdown(s))

In [49]:
def to_lblnewparam(param=None):
    '''
    Convert clirad's input parameter dictionary 
    into something like lblnew's.
    '''
    band = param['band'][0]
    band = mapband_new2old()[band]
    atmpro = param['atmpro']
    tsfc = param['tsfc']
    
    p = {'band': band, 
         'atmpro': atmpro, 'tsfc': tsfc,
         'dv': .001, 'nv': 1000}
    
    if len(param['molecule']) > 1: p['molecule'] = param['molecule']
    else: 
        molecule, conc = list(param['molecule'].items())[0]
        p['molecule'] = molecule
        if conc == 'atmpro': p['conc'] = None
        else: p['conc'] = conc
    return p

def make_query(param=None):
    molecules = {'h2o', 'co2', 'o3', 'n2o', 'ch4'}
    q = {}
    for n, v in param.items():
        if n == 'molecule' and type(v) == dict:
            for mol, conc in v.items():
                q[f'param.molecule.{mol}'] = conc
            for mol in molecules - set(v.keys()):
                q[f'param.molecule.{mol}'] = {'$exists': 0}
        else:
            q[f'param.{n}'] = v
    return q

In [50]:
def subparam_atm_clirad(atmpro='mls'):
    commitnumber = COMMITNUMBER
    atm = nongreys_byband()
    band = list(atm.keys())
    molecule = {mol: conc for _, molconcs in atm.items() 
                for mol, conc in molconcs.items() if mol == MOL}
    return dict(commitnumber=commitnumber,
                band=band, molecule=molecule, 
                atmpro=atmpro, tsfc=ATMPRO[atmpro])

def clirad_data_atm(params_atm=None):
    for p in params_atm:
        q = make_query(p)
        r = client.cliradnew.lw.find_one(q)
        fluxs, cools = [], []
        if not r: print('No data found for', p['molecule'], p['band'])
        else: 
            fluxs.append(load_output_file(io.StringIO(r['output_flux'])))
            cools.append(load_output_file(io.StringIO(r['output_coolr'])))
    return dict(flux=sum(fluxs), cool=sum(cools))

In [51]:
def load_lblnew_data(r):
    '''
    Loads a lblnew-bestfit mongodb document's fluxes and cooling rates.
    '''
    if 'ng_refs' in r['param']:
        ns = {'ds_flux_crd': 'output_flux',
              'ds_cool_crd': 'output_coolr',
              'ds_flux_wgt': 'output_wfluxg',
              'ds_cool_wgt': 'output_wcoolrg'}
    else:
        ns = {'ds_flux_crd': 'output_flux',
              'ds_cool_crd': 'output_coolr',
              'ds_flux_wgt': 'output_wflux',
              'ds_cool_wgt': 'output_wcoolr'}
    return {n: load_output_file(io.StringIO(r[s])) for n, s in ns.items()}

def subparams_atm_lblnew(atmpro='mls'):
    '''
    Returns the list of {band, conc, molecule} dictionaries that
    represent the model atmosphere.
    '''  
    atm = nongreys_byband()
    subparams_clirad = [dict(band=[band], 
                             molecule={mol: conc for mol, conc in molconc.items() if mol == MOL},
                             atmpro=atmpro, tsfc=ATMPRO[atmpro]) 
                        for band, molconc in atm.items() if MOL in molconc]
    return [to_lblnewparam(p) for p in subparams_clirad]

def crd_data_atm(params_atm=None):
    '''
    Returns Dataset containing CRD fluxes and cooling rates at
    different pressures and spectral bands.
    '''
    results_atm = {}
    for p in params_atm:
        if isinstance(p['molecule'], dict): 
            collection = client.lblnew.overlap_lw
        else: 
            collection = client.lblnew.bestfit_lw
        
        q = make_query(p)
        r = collection.find_one(q)
        if not r: print('No data found for', p['molecule'], p['band'])
        else: results_atm[r['param']['band']] = load_lblnew_data(r)

    bands = [mapband_old2new()[b] for b, _ in results_atm.items()]
    fluxs = [d['ds_flux_crd'] for _, d in results_atm.items()]
    cools = [d['ds_cool_crd'] for _, d in results_atm.items()]
    
    d = {}
    d['flux'] = xr.concat(fluxs, dim=bands).rename({'concat_dim': 'band'})
    d['cool'] = xr.concat(cools, dim=bands).rename({'concat_dim': 'band'})
    return d

In [52]:
import rtmtools.clirad.sw.wrangle as cliradwrangle

def oldclirad_data_atm():
    '''
    Load the OLD clirad's results. mls only.
    '''
    fpath = os.path.join('/chia_cluster/home/jackyu/radiation',
                         'clirad-lw',
                         'LW',
                         'examples',
                         'mls75_h2o_atmpro_co2_.0004_o3_atmpro_n2o_3.2e-7_ch4_1.8e-6_H2012',
                         'OUTPUT_CLIRAD.dat')
    
    ds = cliradwrangle.load_OUTPUT_CLIRAD(readfrom=fpath)
    
    ds_cool = xr.Dataset()
    ds_cool.coords['pressure'] = ('pressure', ds['layer_pressure'])
    ds_cool.coords['band'] = ('band', ds['spectral_band'])
    ds_cool['coolrg'] = (('band', 'pressure'), - ds['heating_rate'])
    
    ds_flux = xr.Dataset()
    ds_flux.coords['pressure'] = ('pressure', ds['level_pressure'])
    ds_flux.coords['band'] = ('band', ds['spectral_band'])
    ds_flux['flug'] = (('band', 'pressure'), ds['flux_up'])
    ds_flux['fldg'] = (('band', 'pressure'), ds['flux_down'])
    ds_flux['fnetg'] = (('band', 'pressure'), ds['net_flux'])
    
    
    d = {}
    d['cool'] = ds_cool
    d['flux'] = ds_flux
    return d

In [53]:
def show_makeup():
    df = pd.DataFrame()

    for band, molecule in nongreys_byband().items():
        for name, conc in molecule.items():
            if name == MOL:
                df.loc[name, band] = str(conc)

    df = df.fillna(0)
    df.columns.name = 'clirad band'
    df.index.name = 'molecule'
    
    display.display(df)
    
    display.display(
        display.Markdown('*TABLE.* Non-grey absorbers in the atmosphere.'))


def show_grey_makeup():
    df = pd.DataFrame()
    
    for band, molecule in greyabsorbers_by_band_atm().items():
        if molecule == None:
            pass
        else:
            for name, conc in molecule.items():
                df.loc[name, band] = str(conc)
                
    df = df.fillna(0)
    df.columns.name = 'clirad band'
    df.index.name = 'absorber'
    
    display.display(df)
    display.display(
        display.Markdown('*TABLE.* Grey absorbers in the atmosphere.')
    )

In [54]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
 
def plt_cool_bokeh(pltdata=None, 
                   y_axis_type='linear', prange=(50, 1050)):
    
    ymin = 1e-2 
    ymax = 1020
    
    p2 = figure(y_axis_type=y_axis_type, plot_width=300)
    xmin, xmax = nice_xlims(pltdata, prange=prange)
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                         d['srs'].coords['pressure'].values,
                         color=d['color'], alpha=d['alpha'], 
                         line_width=d['line_width'], 
                         line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, 0))
    legend.label_text_font_size = '8pt'
    p2.add_layout(legend, 'above')
    p2.legend.orientation = 'horizontal'
    p2.legend.location = 'top_center'
    
    return p2

In [55]:
def pltdata_cool(atmpro='mls'):
    ds_clirad = clirad_data_atm(params_atm=[subparam_atm_clirad(atmpro=atmpro)])['cool']
    ds_crd = crd_data_atm(params_atm=subparams_atm_lblnew(atmpro=atmpro))['cool']

    colors = all_palettes['Set1'][4]
    
    data = [
        {'label': 'CLIRAD',
         'srs': fmt_cool(ds_clirad),
         'line_dash': 'dashed', 'line_width': 5,
         'color': colors[0], 'alpha': .6},
        {'label': 'CRD',
         'srs': fmt_cool(ds_crd),
         'line_dash': 'solid', 'line_width': 1.5,
         'marker': 'circle', 'marker_size': 5,
         'color': colors[2], 'alpha': 1}
    ]
    
    # include old CLIRAD's results for mls profile
#    if atmpro == 'mls':
#        d_oldclirad = oldclirad_data_atm()
#        ds_oldclirad = d_oldclirad['cool']        
#        data.append(
#            {'label': 'old CLIRAD (H2012)',
#             'srs': fmt_cool(ds_oldclirad),
#             'line_dash': 'solid', 'line_width': 1.5,
#             'marker': 'square', 'marker_size': 3,
#             'color': colors[3], 'alpha': .5})
    return data


def pltdata_cooldiff(atmpro='mls'):
    ds_clirad = clirad_data_atm(params_atm=[subparam_atm_clirad(atmpro=atmpro)])['cool']
    ds_crd = crd_data_atm(params_atm=subparams_atm_lblnew(atmpro=atmpro))['cool']    
    
    ds_diff = ds_clirad - ds_crd
    
    colors = all_palettes['Set1'][4]
    
    data = [
        {'label': 'CLIRAD - CRD',
         'srs': fmt_cool(ds_diff),
         'line_dash': 'solid', 'line_width': 1.5, 
         'marker': 'circle', 'marker_size': 7,
         'color': colors[3], 'alpha': .8}
    ]
    
    # include old CLIRAD's results for mls profile
#    if atmpro == 'mls':
#        d_oldclirad = oldclirad_data_atm()
#        ds_oldclirad = d_oldclirad['cool']
#        ds_oldclirad.coords['pressure'] = ds_crd.coords['pressure']
#        ds_diff_old = ds_oldclirad.sum('band') - ds_crd.sum('band')
#        data.append(
#            {'label': 'old CLIRAD (H2012) - CRD',
#             'srs': fmt_cool(ds_diff_old),
#             'line_dash': 'dashed', 'line_width': 4,
#             'color': colors[1], 'alpha': .5}
#        )
    return data


def show_cool(atmpro='mls'):
    data_cool = pltdata_cool(atmpro=atmpro)
    p_cool_liny = plt_cool_bokeh(pltdata=data_cool)
    p_cool_logy = plt_cool_bokeh(pltdata=data_cool, 
                                 y_axis_type='log',
                                 prange=(.01, 200))
    
    data_cooldiff = pltdata_cooldiff(atmpro=atmpro)
    p_cooldiff_logy = plt_cool_bokeh(pltdata=data_cooldiff,
                                     y_axis_type='log',
                                     prange=(.01, 200))
    
    everything = gridplot(p_cool_liny, p_cool_logy, 
                          p_cooldiff_logy,
                          ncols=3)
    show(everything)
    display.display(
        display.Markdown('*FIGURE.* Cooling rates & difference.'))
    

In [56]:
def fmt_flux(ds_in):
    ds = ds_in.copy(deep=True)
    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    return ds

def hist_band_vs_flux(da, title='Title'):
    da = da.sel(i=1)
    bands = [str(b.values) for b in da['band']]
    source = ColumnDataSource(data={'band': bands, 'flux': da.values})
    p = figure(x_range=bands, title=title)
    p.vbar(source=source, x='band', top='flux', width=.9)
    p.yaxis.axis_label = 'flux (W m-2)'
    p.xaxis.axis_label = 'spectral band'
    return p

def show_hist_flux(atmpro='mls'):
    ds_crd = crd_data_atm(params_atm=subparams_atm_lblnew(atmpro=atmpro))['flux']
    ds_clirad = clirad_data_atm(params_atm=[subparam_atm_clirad(atmpro=atmpro)])['flux']
    
    ip, varname = 0, 'flug'
    da = (ds_clirad - ds_crd).isel(pressure=ip)[varname]
    p_toa = hist_band_vs_flux(da, title='TOA flux. CLIRAD - CRD.')

    ip, varname = -1, 'fldg'
    da = (ds_clirad - ds_crd).isel(pressure=ip)[varname]
    p_sfc = hist_band_vs_flux(da, title='SFC flux. CLIRAD - CRD.')    
    
    atm_crd = (ds_crd.isel(pressure=0) - ds_crd.isel(pressure=-1))['fnetg']
    atm_clirad = (ds_clirad.isel(pressure=0) - ds_clirad.isel(pressure=-1))['fnetg']
    da = atm_clirad - atm_crd
    p_atm = hist_band_vs_flux(da, title='Atmosphere heating. CLIRAD - CRD.')

    everything = gridplot(p_toa, p_sfc, p_atm, ncols=3, plot_width=300, plot_height=300)
    
    show(everything)
    display.display(
        display.Markdown('*FIGURE.* Difference between CLIRAD and CRD'
          ' in TOA, SFC and net atmosphere flux,'
          ' in each spectral band.'))

def show_tb_flux(atmpro='mls'):
    
    def fmt(da_in):
        da = da_in.copy(deep=True)
        if 'igg' in da.dims:
            da = da.sel(igg=1)
            
        if 'g' in da.dims:
            da = da.sum('g')
        
        if 'band' in da.dims:
            try:
                da = da.squeeze('band')
            except ValueError:
                da = da.sum('band')
                
        return da
    
    ds_crd = crd_data_atm(params_atm=subparams_atm_lblnew(atmpro=atmpro))['flux']
    olr_crd = ds_crd['flug'].isel(pressure=0)
    sfc_crd = ds_crd['fldg'].isel(pressure=-1)
    atm_crd = (ds_crd.isel(pressure=0) - ds_crd.isel(pressure=-1))['fnetg']
    
    ds_clirad = clirad_data_atm(params_atm=[subparam_atm_clirad(atmpro=atmpro)])['flux']
    olr_clirad = ds_clirad['flug'].isel(pressure=0)
    sfc_clirad = ds_clirad['fldg'].isel(pressure=-1)
    atm_clirad = (ds_clirad.isel(pressure=0) - ds_clirad.isel(pressure=-1))['fnetg']
    
#    if atmpro == 'mls':
#        ds_oldclirad = oldclirad_data_atm()['flux']
#        ds_oldclirad['pressure'] = ds_crd['pressure']
#        olr_oldclirad = ds_oldclirad['flug'].isel(pressure=0)
#        sfc_oldclirad = ds_oldclirad['fldg'].isel(pressure=-1)
#        atm_oldclirad = (ds_oldclirad.isel(pressure=0)
#                         - ds_oldclirad.isel(pressure=-1))['fnetg']
        
    df = pd.DataFrame()
    df.index.name = 'Sum over bands'
    
#    if atmpro == 'mls':
#        df.loc['old CLIRAD - CRD', 'OLR flux'] = (fmt(olr_oldclirad) - fmt(olr_crd)).values
#        df.loc['old CLIRAD - CRD', 'SFC flux'] = (fmt(sfc_oldclirad) - fmt(sfc_crd)).values
#        df.loc['old CLIRAD - CRD', 'ATM heating'] = (fmt(atm_oldclirad) - fmt(atm_crd)).values
    
    df.loc['CLIRAD - CRD', 'OLR flux'] = (fmt(olr_clirad) - fmt(olr_crd)).values
    df.loc['CLIRAD - CRD', 'SFC flux'] = (fmt(sfc_clirad) - fmt(sfc_crd)).values
    df.loc['CLIRAD - CRD', 'ATM heating'] = (fmt(atm_clirad) - fmt(atm_crd)).values

    df.loc['CRD', 'OLR flux'] = fmt(olr_crd).values
    df.loc['CRD', 'SFC flux'] = fmt(sfc_crd).values
    df.loc['CRD', 'ATM heating'] = fmt(atm_crd).values
        
    df = df.astype('float').round(2)
    
    display.display(df)
    display.display(
        display.Markdown('*TABLE.* Difference between CLIRAD and CRD'
          ' in TOA, SFC and net atmosphere flux,'
          ' over all spectral bands. CRD\'s'
          ' TOA, SFC and net atmosphere flux,'
          ' over all spectral bands.'))

In [57]:
def cool_tofile(atmpro=None):
    dcli = clirad_data_atm(params_atm=[subparam_atm_clirad(atmpro=atmpro)])
    dcrd = crd_data_atm(params_atm=subparams_atm_lblnew(atmpro=atmpro))
    
    dcli = dcli['cool'].sel(i=1).drop('i').sum('band')
    dcrd = dcrd['cool'].sum('band')
    
    dcli = dcli['coolrg'].to_dataframe()
    dcrd = dcrd['coolrg'].to_dataframe()
    
    dcli = dcli.set_index('layer', append=True)
    dcrd = dcrd.set_index('layer', append=True)
    
    ddif = dcli - dcrd
    
    df = pd.concat([dcrd, dcli, ddif], axis=1, keys=['CRD', 'CLIRAD', 'CLIRAD - CRD'])
    return df


def flux_tofile(atmpro=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    ilevels = [0, tropopauses[atmpro], -1]
    
    dcli = clirad_data_atm(params_atm=[subparam_atm_clirad(atmpro=atmpro)])
    dcrd = crd_data_atm(params_atm=subparams_atm_lblnew(atmpro=atmpro))
    
    dcli = dcli['flux'].sel(i=1).drop('i').sum('band').isel(pressure=ilevels)
    dcrd = dcrd['flux'].sum('band').isel(pressure=ilevels)
    
    dcli = dcli.to_dataframe()
    dcrd = dcrd.to_dataframe()
    
    dcli = dcli.set_index('level', append=True)
    dcrd = dcrd.set_index('level', append=True)
    
    ddif = dcli - dcrd
    
    df = pd.concat([dcrd, dcli, ddif], axis=1, keys=['CRD', 'CLIRAD', 'CLIRAD - CRD'])
    return df


def script_tofile():
    fname = f'results_{MOL}atm_cliradsinglerun-Copy2.xlsx'
    writer = pd.ExcelWriter(fname)
    for atmpro in ('mls', 'saw', 'trp'):
        dflux = flux_tofile(atmpro=atmpro)
        dflux.to_excel(writer, f'flux {atmpro}')
        
        dcool = cool_tofile(atmpro=atmpro)
        dcool.to_excel(writer, f'cool {atmpro}')
        
    html = f'Download: <a href="./{fname}">{fname}</a>'
    show_html(html)

        

In [58]:
def script():
    
    d_atm = {'mls': 'mid-latitude summer',
             'saw': 'sub-arctic winter',
             'trp': 'tropical'}
    
    title = ('## Results over entire range of molecules'
             ' and spectral bands')
    
    s_makeup = 'Makeup of atmosphere.'
    s_atmpro = '# {}'
    s_cool = 'Cooling rates. {}.'
    s_flux = 'Fluxes. {}.'
    
    atmpros = ['mls', 'saw', 'trp']
    
    # TOC
    show_markdown(title)
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_makeup))
    for atmpro in atmpros:
        show_markdown('**' + d_atm[atmpro] + '**')
        show_html(climahtml.getHTML_hrefanchor(s_cool.format(atmpro)))
        show_html(climahtml.getHTML_hrefanchor(s_flux.format(atmpro)))

        
    # Atmosphere makeup
    show_html(climahtml.getHTML_idanchor(s_makeup))
    show_markdown(climahtml.getMarkdown_sectitle(s_makeup))
    show_makeup()
#    show_grey_makeup()
        
    for atmpro in atmpros:
        show_html(climahtml.getHTML_idanchor(s_cool.format(atmpro)))
        show_markdown(
            climahtml.getMarkdown_sectitle(s_cool.format(atmpro)))
        show_cool(atmpro=atmpro)
        show_html(climahtml.getHTML_idanchor(s_flux.format(atmpro)))
        show_markdown(
            climahtml.getMarkdown_sectitle(s_flux.format(atmpro)))
        show_hist_flux(atmpro=atmpro)
        show_tb_flux(atmpro=atmpro)
    
    script_tofile()
    
script()  

## Results over entire range of molecules and spectral bands

### Table of Contents

**mid-latitude summer**

**sub-arctic winter**

**tropical**

...

*FIGURE.* Cooling rates & difference.

# Fluxes. trp.

*FIGURE.* Difference between CLIRAD and CRD in TOA, SFC and net atmosphere flux, in each spectral band.

,OLR flux,SFC flux,ATM heating
Sum over bands,,,
CLIRAD - CRD,-227.54,0.67,0.01
CRD,-182.34,85.22,-36.52


*TABLE.* Difference between CLIRAD and CRD in TOA, SFC and net atmosphere flux, over all spectral bands. CRD's TOA, SFC and net atmosphere flux, over all spectral bands.

In [59]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')